In [1]:
import requests
import pandas as pd
import numpy as np
import pickle

In [2]:
def save_pkl(file_dir, data):
    f = open(file_dir,"wb")
    pickle.dump(data, f, protocol=4)
    f.close()
    
def read_pkl(file_dir):
    f = open(file_dir,"rb")
    data = pickle.load(f)
    return data

In [3]:
df = pd.read_csv("London/London_historical_meo_grid.csv")
df.rename(columns={'utc_time': 'time',
                   'wind_speed/kph': 'wind_speed'
                  }, inplace=True)
df['time'] = pd.to_datetime(df['time'])
df.index = df['time']
df = df.sort_index()
group = df.groupby("stationName")
meo_group = {}
for name, g in group:
    meo_group[name] = g.sort_index()
save_pkl("London/meo_train.pkl", meo_group)
df.tail(3)

,stationName,longitude,latitude,time,temperature,pressure,humidity,wind_direction,wind_speed
time,,,,,,,,,
2018-03-27 05:00:00,london_grid_858,2.0,52.3,2018-03-27 05:00:00,6.91,1011.01,85.91,189.05,25.75
2018-03-27 05:00:00,london_grid_859,2.0,52.4,2018-03-27 05:00:00,6.79,1010.92,85.94,188.15,24.79
2018-03-27 05:00:00,london_grid_860,2.0,52.5,2018-03-27 05:00:00,6.74,1010.89,85.95,187.84,24.47


In [4]:
meo_group = read_pkl("London/meo_train.pkl")

In [5]:
nearst = {
    'GB0': 'london_grid_451', 'CD1': 'london_grid_388', 'HV1': 'london_grid_472', 'CD9': 'london_grid_409',
    'TD5': 'london_grid_366', 'GR4': 'london_grid_451', 'RB7': 'london_grid_452', 'TH4': 'london_grid_430',
    'HR1': 'london_grid_368', 'BL0': 'london_grid_409', 'GR9': 'london_grid_430', 'ST5': 'london_grid_408',
    'LH0': 'london_grid_346', 'KF1': 'london_grid_388', 'MY7': 'london_grid_388', 'LW2': 'london_grid_430',
    'GN3': 'london_grid_451', 'GN0': 'london_grid_451', 'CT3': 'london_grid_409', 'CR8': 'london_grid_408',
    'BX1': 'london_grid_472' # ,'CT2': 'london_grid_409'
}

In [6]:
# read the original features (original version of station_group) from air_processing.pkl

station_group = read_pkl("London/air_processing.pkl")
# for name in station_group:
#     print(station_group[name])
station_position = read_pkl("London/station_processing.pkl")
# station_position

In [8]:
# merge the weather condition features
attr_need = ["temperature", "pressure", "humidity", "wind_direction", "wind_speed"]
for name in station_group:
    # print(name)
    l = len(station_group[name])
    grid_id = nearst[name]
    for attr in attr_need:
        keep_values = meo_group[grid_id][attr].values
        extra_array = np.zeros(l - len(keep_values))
        station_group[name][attr] = np.append(keep_values, extra_array)
#     print(station_group[name])
station_group['ST5']

,station_id,PM25_Concentration,PM10_Concentration,NO2_Concentration,time_year,time_month,time_week,time_day,time_hour,temperature,pressure,humidity,wind_direction,wind_speed
time,,,,,,,,,,,,,,
2017-01-01 00:00:00,ST5,25.0,29.0,14.5,2017.0,1.0,6.0,1.0,0.0,5.54,1015.62,92.97,222.20,14.12
2017-01-01 01:00:00,ST5,21.0,25.0,24.0,2017.0,1.0,6.0,1.0,1.0,5.58,1014.43,91.23,219.58,15.24
2017-01-01 02:00:00,ST5,20.0,23.0,28.1,2017.0,1.0,6.0,1.0,2.0,5.63,1013.24,89.49,217.33,16.39
2017-01-01 03:00:00,ST5,22.0,21.0,20.6,2017.0,1.0,6.0,1.0,3.0,5.68,1012.05,87.74,215.37,17.56
2017-01-01 04:00:00,ST5,22.0,26.0,34.6,2017.0,1.0,6.0,1.0,4.0,5.86,1011.11,88.39,213.93,17.81
2017-01-01 05:00:00,ST5,24.0,29.0,23.8,2017.0,1.0,6.0,1.0,5.0,6.05,1010.17,89.04,212.53,18.08
2017-01-01 06:00:00,ST5,16.0,23.0,17.2,2017.0,1.0,6.0,1.0,6.0,6.23,1009.22,89.68,211.18,18.36
2017-01-01 07:00:00,ST5,20.0,27.0,15.2,2017.0,1.0,6.0,1.0,7.0,6.49,1008.65,90.45,209.15,17.95
2017-01-01 08:00:00,ST5,20.0,25.0,20.6,2017.0,1.0,6.0,1.0,8.0,6.74,1008.08,91.22,207.03,17.57


In [9]:
# merge the holiday condition feature
df_h = pd.read_csv("London/holiday_ld.csv")
holiday_dict = {}
date_list = df_h['date'].values
holiday_list = df_h['holiday'].values
for i in range(len(date_list)):
    holiday_dict[str(date_list[i])] = holiday_list[i]
for name in station_group:
    # print(name)
    time_list = station_group[name].index.values
    temp_list = []
    for time in time_list:
        time = str(time)
        date = time[:4]+time[5:7]+time[8:10]
        temp_list.append(holiday_dict[date])
    station_group[name]['holiday'] = temp_list
#     print(station_group[name])
station_group['ST5']

,station_id,PM25_Concentration,PM10_Concentration,NO2_Concentration,time_year,time_month,time_week,time_day,time_hour,temperature,pressure,humidity,wind_direction,wind_speed,holiday
time,,,,,,,,,,,,,,,
2017-01-01 00:00:00,ST5,25.0,29.0,14.5,2017.0,1.0,6.0,1.0,0.0,5.54,1015.62,92.97,222.20,14.12,2
2017-01-01 01:00:00,ST5,21.0,25.0,24.0,2017.0,1.0,6.0,1.0,1.0,5.58,1014.43,91.23,219.58,15.24,2
2017-01-01 02:00:00,ST5,20.0,23.0,28.1,2017.0,1.0,6.0,1.0,2.0,5.63,1013.24,89.49,217.33,16.39,2
2017-01-01 03:00:00,ST5,22.0,21.0,20.6,2017.0,1.0,6.0,1.0,3.0,5.68,1012.05,87.74,215.37,17.56,2
2017-01-01 04:00:00,ST5,22.0,26.0,34.6,2017.0,1.0,6.0,1.0,4.0,5.86,1011.11,88.39,213.93,17.81,2
2017-01-01 05:00:00,ST5,24.0,29.0,23.8,2017.0,1.0,6.0,1.0,5.0,6.05,1010.17,89.04,212.53,18.08,2
2017-01-01 06:00:00,ST5,16.0,23.0,17.2,2017.0,1.0,6.0,1.0,6.0,6.23,1009.22,89.68,211.18,18.36,2
2017-01-01 07:00:00,ST5,20.0,27.0,15.2,2017.0,1.0,6.0,1.0,7.0,6.49,1008.65,90.45,209.15,17.95,2
2017-01-01 08:00:00,ST5,20.0,25.0,20.6,2017.0,1.0,6.0,1.0,8.0,6.74,1008.08,91.22,207.03,17.57,2


In [10]:
# merge the feature of station index

PM25_mean_list = []

for name in station_group:
    PM25_mean = np.mean(station_group[name]['PM25_Concentration'].values)
    PM25_mean_list.append(PM25_mean)
        
for name in station_group:
    small_time = 0
    large_time = 0
    PM25_mean = np.mean(station_group[name]['PM25_Concentration'].values)
    for i in range(len(PM25_mean_list)):
        if PM25_mean_list[i] > PM25_mean:
            small_time = small_time + 1
        else:
            large_time = large_time + 1
            
    l = len(station_group[name])        
    station_group[name]['station_index'] = np.full((l), small_time)
    
station_group['ST5']

,station_id,PM25_Concentration,PM10_Concentration,NO2_Concentration,time_year,time_month,time_week,time_day,time_hour,temperature,pressure,humidity,wind_direction,wind_speed,holiday,station_index
time,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,ST5,25.0,29.0,14.5,2017.0,1.0,6.0,1.0,0.0,5.54,1015.62,92.97,222.20,14.12,2,4
2017-01-01 01:00:00,ST5,21.0,25.0,24.0,2017.0,1.0,6.0,1.0,1.0,5.58,1014.43,91.23,219.58,15.24,2,4
2017-01-01 02:00:00,ST5,20.0,23.0,28.1,2017.0,1.0,6.0,1.0,2.0,5.63,1013.24,89.49,217.33,16.39,2,4
2017-01-01 03:00:00,ST5,22.0,21.0,20.6,2017.0,1.0,6.0,1.0,3.0,5.68,1012.05,87.74,215.37,17.56,2,4
2017-01-01 04:00:00,ST5,22.0,26.0,34.6,2017.0,1.0,6.0,1.0,4.0,5.86,1011.11,88.39,213.93,17.81,2,4
2017-01-01 05:00:00,ST5,24.0,29.0,23.8,2017.0,1.0,6.0,1.0,5.0,6.05,1010.17,89.04,212.53,18.08,2,4
2017-01-01 06:00:00,ST5,16.0,23.0,17.2,2017.0,1.0,6.0,1.0,6.0,6.23,1009.22,89.68,211.18,18.36,2,4
2017-01-01 07:00:00,ST5,20.0,27.0,15.2,2017.0,1.0,6.0,1.0,7.0,6.49,1008.65,90.45,209.15,17.95,2,4
2017-01-01 08:00:00,ST5,20.0,25.0,20.6,2017.0,1.0,6.0,1.0,8.0,6.74,1008.08,91.22,207.03,17.57,2,4


In [16]:
# merge the feature of pressure/temperature，temperature/humidity，PM25+PM10，PM25/PM10

for name in station_group:
    df = station_group[name]
    df['pressure_temperature'] = df.apply(lambda x: x['pressure'] * x['temperature'], axis=1)
    df['temperature_humidity'] = df.apply(lambda x: x['temperature'] / (x['humidity']+0.01), axis=1)
    df['PM'] = df.apply(lambda x: x['PM25_Concentration'] + x['PM10_Concentration'], axis=1)  
    df['PM25_10'] = df.apply(lambda x: x['PM25_Concentration'] / (x['PM10_Concentration']+0.01), axis=1)
    
station_group['ST5']

,station_id,PM25_Concentration,PM10_Concentration,NO2_Concentration,time_year,time_month,time_week,time_day,time_hour,temperature,pressure,humidity,wind_direction,wind_speed,holiday,station_index,pressure_temperature,temperature_humidity,PM,PM25_10
time,,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,ST5,25.0,29.0,14.5,2017.0,1.0,6.0,1.0,0.0,5.54,1015.62,92.97,222.20,14.12,2,4,5626.5348,0.059583,54.0,0.861772
2017-01-01 01:00:00,ST5,21.0,25.0,24.0,2017.0,1.0,6.0,1.0,1.0,5.58,1014.43,91.23,219.58,15.24,2,4,5660.5194,0.061157,46.0,0.839664
2017-01-01 02:00:00,ST5,20.0,23.0,28.1,2017.0,1.0,6.0,1.0,2.0,5.63,1013.24,89.49,217.33,16.39,2,4,5704.5412,0.062905,43.0,0.869187
2017-01-01 03:00:00,ST5,22.0,21.0,20.6,2017.0,1.0,6.0,1.0,3.0,5.68,1012.05,87.74,215.37,17.56,2,4,5748.4440,0.064729,43.0,1.047120
2017-01-01 04:00:00,ST5,22.0,26.0,34.6,2017.0,1.0,6.0,1.0,4.0,5.86,1011.11,88.39,213.93,17.81,2,4,5925.1046,0.066290,48.0,0.845829
2017-01-01 05:00:00,ST5,24.0,29.0,23.8,2017.0,1.0,6.0,1.0,5.0,6.05,1010.17,89.04,212.53,18.08,2,4,6111.5285,0.067939,53.0,0.827301
2017-01-01 06:00:00,ST5,16.0,23.0,17.2,2017.0,1.0,6.0,1.0,6.0,6.23,1009.22,89.68,211.18,18.36,2,4,6287.4406,0.069461,39.0,0.695350
2017-01-01 07:00:00,ST5,20.0,27.0,15.2,2017.0,1.0,6.0,1.0,7.0,6.49,1008.65,90.45,209.15,17.95,2,4,6546.1385,0.071744,47.0,0.740466
2017-01-01 08:00:00,ST5,20.0,25.0,20.6,2017.0,1.0,6.0,1.0,8.0,6.74,1008.08,91.22,207.03,17.57,2,4,6794.4592,0.073879,45.0,0.799680


In [18]:
df = station_group['ST5']
group = df.groupby("time_hour")
hour_group = {}
for hour, g in group:
    hour_group[str(hour)] = g.sort_index()
for hour in hour_group:
    hour_df = hour_group[hour]
    print(hour_df)

           station_id  PM25_Concentration  PM10_Concentration  \
time                                                            
2017-01-01        ST5                25.0                29.0   
2017-01-02        ST5                 7.0                12.0   
2017-01-03        ST5                20.0                26.0   
2017-01-04        ST5                 9.0                10.0   
2017-01-05        ST5                17.0                24.0   
2017-01-06        ST5                41.0                55.0   
2017-01-07        ST5                 9.0                 6.0   
2017-01-08        ST5                27.0                31.0   
2017-01-09        ST5                12.0                15.0   
2017-01-10        ST5                12.0                16.0   
2017-01-11        ST5                11.0                20.0   
2017-01-12        ST5                 7.0                23.0   
2017-01-13        ST5                 6.0                 5.0   
2017-01-14        ST5    

In [31]:

for name in station_group:
    df = station_group[name]
    PM25_list = df['PM25_Concentration'].values
    PM10_list = df['PM10_Concentration'].values
    NO2_list = df['NO2_Concentration'].values
    temperature_list = df['temperature'].values
    pressure_list = df['pressure'].values
    humidity_list = df['humidity'].values
    wind_speed_list = df['wind_speed'].values
    wind_direction_list = df['wind_direction'].values

    hour_number = 14 * 24

    mean_PM25_list = []
    mean_PM10_list = []
    mean_NO2_list = []
    mean_temperature_list = []
    mean_pressure_list = []
    mean_humidity_list = []
    mean_wind_speed_list = []
    mean_wind_direction_list = []
    max_PM25_list = []
    max_PM10_list = []
    max_NO2_list = []
    max_temperature_list = []
    max_pressure_list = []
    max_humidity_list = []
    max_wind_speed_list = []
    var_PM25_list = []
    var_PM10_list = []
    var_NO2_list = []
    var_temperature_list = []
    var_pressure_list = []
    var_humidity_list = []
    var_wind_speed_list = []
    var_wind_direction_list = []
    for i in range(hour_number-1):
        mean_PM25_list.append(0)
        mean_PM10_list.append(0)
        mean_NO2_list.append(0)
        mean_temperature_list.append(0)
        mean_pressure_list.append(0)
        mean_humidity_list.append(0)
        mean_wind_speed_list.append(0)
        mean_wind_direction_list.append(0)
        max_PM25_list.append(0)
        max_PM10_list.append(0)
        max_NO2_list.append(0)
        max_temperature_list.append(0)
        max_pressure_list.append(0)
        max_humidity_list.append(0)
        max_wind_speed_list.append(0)
        var_PM25_list.append(0)
        var_PM10_list.append(0)
        var_NO2_list.append(0)
        var_temperature_list.append(0)
        var_pressure_list.append(0)
        var_humidity_list.append(0)
        var_wind_speed_list.append(0)
        var_wind_direction_list.append(0)

    for i in range(hour_number-1, len(PM25_list)):
        PM25_array = PM25_list[i-hour_number+1:i+1]
        PM10_array = PM10_list[i-hour_number+1:i+1]
        NO2_array = NO2_list[i-hour_number+1:i+1]
        temperature_array = temperature_list[i-hour_number+1:i+1]
        pressure_array = pressure_list[i-hour_number+1:i+1]
        humidity_array = humidity_list[i-hour_number+1:i+1]
        wind_speed_array = wind_speed_list[i-hour_number+1:i+1]
        wind_direction_array = wind_direction_list[i-hour_number+1:i+1]

        mean_PM25_list.append(np.mean(PM25_array))
        mean_PM10_list.append(np.mean(PM10_array))
        mean_NO2_list.append(np.mean(NO2_array))
        mean_temperature_list.append(np.mean(temperature_array))
        mean_pressure_list.append(np.mean(pressure_array))
        mean_humidity_list.append(np.mean(humidity_array))
        mean_wind_speed_list.append(np.mean(wind_speed_array))
        mean_wind_direction_list.append(np.mean(wind_direction_array))
        max_PM25_list.append(np.max(PM25_array))
        max_PM10_list.append(np.max(PM10_array))
        max_NO2_list.append(np.max(NO2_array))
        max_temperature_list.append(np.max(temperature_array))
        max_pressure_list.append(np.max(pressure_array))
        max_humidity_list.append(np.max(humidity_array))
        max_wind_speed_list.append(np.max(wind_speed_array))
        var_PM25_list.append(np.var(PM25_array))
        var_PM10_list.append(np.var(PM10_array))
        var_NO2_list.append(np.var(NO2_array))
        var_temperature_list.append(np.var(temperature_array))
        var_pressure_list.append(np.var(pressure_array))
        var_humidity_list.append(np.var(humidity_array))
        var_wind_speed_list.append(np.var(wind_speed_array))
        var_wind_direction_list.append(np.var(wind_direction_array))
    #     print(np.mean(array))
    #     print(np.max(array))
    #     print(np.var(array))

    df['mean_PM25'] = mean_PM25_list
    df['mean_PM10'] = mean_PM10_list
    df['mean_NO2'] = mean_NO2_list
    df['mean_temperature'] = mean_temperature_list
    df['mean_pressure'] = mean_pressure_list
    df['mean_humidity'] = mean_humidity_list
    df['mean_wind_speed'] = mean_wind_speed_list
    df['mean_wind_direction'] = mean_wind_direction_list
    df['max_PM25'] = max_PM25_list
    df['max_PM10'] = max_PM10_list
    df['max_NO2'] = max_NO2_list
    df['max_temperature'] = max_temperature_list
    df['max_pressure'] = max_pressure_list
    df['max_humidity'] = max_humidity_list
    df['max_wind_speed'] = max_wind_speed_list
    df['var_PM25'] = var_PM25_list
    df['var_PM10'] = var_PM10_list
    df['var_NO2'] = var_NO2_list
    df['var_temperature'] = var_temperature_list
    df['var_pressure'] = var_pressure_list
    df['var_humidity'] = var_humidity_list
    df['var_wind_speed'] = var_wind_speed_list
    df['var_wind_speed'] = var_wind_direction_list

In [32]:
station_group['ST5']

,station_id,PM25_Concentration,PM10_Concentration,NO2_Concentration,time_year,time_month,time_week,time_day,time_hour,temperature,...,max_humidity,max_wind_speed,var_PM25,var_PM10,var_NO2,var_temperature,var_pressure,var_humidity,var_wind_speed,var_wind_direction
time,,,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,ST5,25.0,29.0,14.5,2017.0,1.0,6.0,1.0,0.0,5.54,...,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-01-01 01:00:00,ST5,21.0,25.0,24.0,2017.0,1.0,6.0,1.0,1.0,5.58,...,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-01-01 02:00:00,ST5,20.0,23.0,28.1,2017.0,1.0,6.0,1.0,2.0,5.63,...,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-01-01 03:00:00,ST5,22.0,21.0,20.6,2017.0,1.0,6.0,1.0,3.0,5.68,...,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-01-01 04:00:00,ST5,22.0,26.0,34.6,2017.0,1.0,6.0,1.0,4.0,5.86,...,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-01-01 05:00:00,ST5,24.0,29.0,23.8,2017.0,1.0,6.0,1.0,5.0,6.05,...,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-01-01 06:00:00,ST5,16.0,23.0,17.2,2017.0,1.0,6.0,1.0,6.0,6.23,...,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-01-01 07:00:00,ST5,20.0,27.0,15.2,2017.0,1.0,6.0,1.0,7.0,6.49,...,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-01-01 08:00:00,ST5,20.0,25.0,20.6,2017.0,1.0,6.0,1.0,8.0,6.74,...,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [46]:
def show(key):
    s = ['BL0','CD1','CD9','GN0','GN3','GR4','GR9','HV1','KF1','LW2','MY7','ST5','TH4']
    for name in station_group:
        if name in s:
            df = station_group[name]
            temp = df[key].values
            t = []
            for i in range(len(temp)-1):
                t.append(abs(temp[i]-temp[i+1]))
    print(np.mean(t))
show('PM25_Concentration')
show('PM10_Concentration')
show('NO2_Concentration')
show('temperature')
show('pressure')
show('humidity')
show('wind_speed')
show('wind_direction')

2.308518806783941
4.084316044144522
7.801108166176109
0.46577918502202637
0.43580488252569755
2.0094603524229075
0.8710737885462555
7.655961820851688


In [33]:
# save all of the features (final version of station_group) into station_final.pkl
save_pkl("London/station_final.pkl", station_group)

In [ ]:
# load the file
station_group = read_pkl("London/station_final.pkl")
station_group['ST5']